---
title: "MyQLM tutorial for gate-based simulations"
author: "Mi-Song Dupuy"
date: "2025-07-02"
format:
  html:
    code-fold: true
jupyter: python3.12
---

In this tutorial, we will give the basics to write a gate-based quantum circuit and its execution using the MyQLM package.

# Installation 

MyQLM is available as a package via `pip` on **Python 3.12**.

# MyQLM simulation: a quick tutorial

To emulate a quantum circuit, you first create a `Program`.

In [ ]:
from qat.lang.AQASM import Program
my_program = Program()

## Quantum register

You need to specify the number of qubits in the register associated to `my_program`.

In [ ]:
qregister = my_program.qalloc(2) #allocates 2 qubits

`qregister` behaves as a list of qubits (here of 2 qubits). By default, the quantum register is initialised at the state $\vert 0\rangle$.

## Application of gates

We can apply quantum gates on qubits of the quantum register.

In [ ]:
from qat.lang.AQASM import H, CNOT #import the Hadamard gate and the CNOT gate
H(qregister[0])
CNOT(qregister) #qregister has 2 qubits so it works

## Circuit visualisation 

The circuit can be extracted from the `Program` using the `to_circ` method and then displayed.

In [ ]:
circuit = my_program.to_circ()
circuit.display()

## Concatenation of quantum circuits

The concatenation (or composition) of quantum circuits is done by the operation `+`.

In [ ]:
circuit = circuit + circuit # applies the circuit twice 
circuit.display()

## Circuit execution 

For noiseless simulations, there are two ways to perform the execution.

First you need to create a job from a circuit.

In [ ]:
my_program_job = circuit.to_job()

### Exact calculation

In this case, the result of the circuit is computed and the code gives the exact amplitudes of the result of the circuit.

In [ ]:
from qat.qpus import PyLinalg 
linalgqpu = PyLinalg()
result = linalgqpu.submit(my_program_job) #compute the result of the linear algebra
for sample in result:
    print("State %s amplitude %s" % (sample.state, sample.amplitude))

### Emulation of a QPU

It is also possible to *emulate* the behaviour of a QPU by specifying the number of *shots* or *samples* (*i.e.* the number of times the quantum circuit is measured).

In [ ]:
my_program_job2 = circuit.to_job(nbshots=1000) #10 shots are executed
result = linalgqpu.submit(my_program_job2)
for sample in result:
    print("We measured the state {} (its probability is {} and its amplitude {})".format(sample.state, sample.probability, sample.amplitude)) #the amplitude is not returned

# MyQLM features

## Partial measurements

Partial measurements on a subset of the qubits of the register by using `measure`.
By default, `measure` projects the corresponding qubits to eigenstates of the $\hat{S_z}$ operator, *i.e.* to $|\mu_1 \dots \mu_n\rangle$ where $\mu_i = 0$ or 1.

In the circuit below, a measurement is performed at the end of the circuit. 

In [ ]:
deterministic = Program()
deterministic_qubits = deterministic.qalloc(1)
H(deterministic_qubits)
H(deterministic_qubits) #second application of the Hadamard gate
deterministic.measure([0]) #applies a measurement only to the 1st qbit
deterministic_circuit = deterministic.to_circ()
deterministic_circuit.display()
deterministic_job = deterministic_circuit.to_job(nbshots=1000) 
result = linalgqpu.submit(deterministic_job)
for sample in result:
    print("We measured the state {} (its probability is {} and its amplitude {})".format(sample.state, sample.probability, sample.amplitude)) #we only get \vert 0>

In the example below, a measurement is performed between both Hadamard gate applications. After the first Hadamard gate, the state is projected to $\vert 0\rangle$ or $\vert 1\rangle$, thus the output of the whole circuit is a superposition of $\vert 0\rangle$ or $\vert 1\rangle$.

In [ ]:
uniform = Program()
uniform_qubits = uniform.qalloc(1)
H(uniform_qubits)
uniform.measure([0]) #applies a measurement only to the 1st qbit
H(uniform_qubits)
uniform_circuit = uniform.to_circ()
uniform_circuit.display()
uniform_job = uniform_circuit.to_job(nbshots=1000) 
result = linalgqpu.submit(uniform_job)
for sample in result:
    print("We measured the state {} (its probability is {} and its amplitude {})".format(sample.state, sample.probability, sample.amplitude)) #we only get |0> and | 1> with probability 1/2

In the example below, the incorporation of a controlled gate influences the output of the measure of the first qubit.

In [ ]:
deferred = Program()
deferred_qubits = deferred.qalloc(2)
H(deferred_qubits[0])
CNOT(deferred_qubits)
H(deferred_qubits[0])
deferred.measure([0]) #applies a measurement only to the 1st qbit
deferred_circuit = deferred.to_circ()
deferred_circuit.display()
deferred_job = deferred_circuit.to_job(nbshots=1000) 
result = linalgqpu.submit(deferred_job)
for sample in result:
    print("We measured the state {} (its probability is {} and its amplitude {})".format(sample.state, sample.probability, sample.amplitude)) #we get approximately 1/4 for each result

## List of common gates

### Constant gates

| Function | Description | Matrix Representation | Usage |
|----------|-------------|-----------------------|-------|
| X | Pauli-X gate, NOT gate | $\begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$ | `X(qubit)` |
| Y | Pauli-Y gate | $\begin{bmatrix} 0 & -i \\ i & 0 \end{bmatrix}$ | `Y(qubit)` |
| Z | Pauli-Z gate | $\begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}$ | `Z(qubit)` |
| H | Hadamard gate | $\begin{bmatrix} \frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}} \end{bmatrix}$ | `H(qubit)` |
| S | Phase gate (or S gate) | $\begin{bmatrix} 1 & 0 \\ 0 & i \end{bmatrix}$ | `S(qubit)` |
| T | T gate | $\begin{bmatrix} 1 & 0 \\ 0 & e^{i\pi/4} \end{bmatrix}$ | `T(qubit)` |
| CNOT | CNOT (Controlled NOT) gate | $\begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix}$ | `CNOT(control_qubit, target_qubit)` |
| CCNOT | Toffoli gate (or CCNOT gate). | $\begin{bmatrix} 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\ 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \end{bmatrix}$ | `CCNOT(control_qubit1, control_qubit2, target_qubit)` |
| CSIGN | Controlled Sign or C-Z gate | $\begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & -1 \end{bmatrix}$ | `CSIGN(control_qubit, target_qubit)` |
| SWAP | SWAP gate | $\begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}$ | `SWAP(qubit1, qubit2)` |
| SQRTSWAP | Square Root of SWAP gate. It creates a superposition of swapped and non-swapped states. | $\begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & \frac{1}{2}(1+i) & \frac{1}{2}(1-i) & 0 \\ 0 & \frac{1}{2}(1-i) & \frac{1}{2}(1+i) & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}$ | `SQRTSWAP(qubit1, qubit2)` |
| ISWAP | iSWAP gate. It swaps the states of two qubits with a phase factor of i. | $\begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 0 & i & 0 \\ 0 & i & 0 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}$ | `ISWAP(qubit1, qubit2)` |


### Parametrised gates 

| Function | Description | Matrix Representation | Usage |
|----------|-------------|-----------------------|-------|
| RX(θ) | Rotation around the X-axis by an angle θ. | $\begin{bmatrix} \cos(\theta/2) & -i\sin(\theta/2) \\ -i\sin(\theta/2) & \cos(\theta/2) \end{bmatrix}$ | `RX(theta)(qubit)` |
| RY(θ) | Rotation around the Y-axis by an angle θ. | $\begin{bmatrix} \cos(\theta/2) & -\sin(\theta/2) \\ \sin(\theta/2) & \cos(\theta/2) \end{bmatrix}$ | `RY(theta)(qubit)` |
| RZ(θ) | Rotation around the Z-axis by an angle θ. | $\begin{bmatrix} e^{-i\theta/2} & 0 \\ 0 & e^{i\theta/2} \end{bmatrix}$ | `RZ(theta)(qubit)` |
| PH(φ) | Phase gate that leaves $\vert 0\rangle$ unchanged and maps $\vert 1\rangle$ to $e^{iφ}\vert 1\rangle$. | $\begin{bmatrix} 1 & 0 \\ 0 & e^{iφ} \end{bmatrix}$ | `PH(phi)(qubit)` |


### Create a custom control gate

Custom control gates can be defined using the `.ctrl()` method.

In [ ]:
from qat.lang.AQASM import Y

CY_test = Program()
qubits = CY_test.qalloc(2)
Y.ctrl()(qubits) #C-Y gate
CY_test.to_circ().display()

It also works with parametrised gates.

In [ ]:
from qat.lang.AQASM import PH

CPH_test = Program()
qubits = CPH_test.qalloc(2)
PH(0.5).ctrl()(qubits) #C-PH gate
CPH_test.to_circ().display()

### Create a general custom gate (advanced)

If the custom gate is composed of common gates, it is possible to use `QRoutine` in order to define a subcircuit that would behave as a gate.

In [ ]:
from qat.lang.AQASM import QRoutine

def hadamard_generator(n):
    routine = QRoutine() #QRoutine creates subcircuits that behave as a Gate object.
    wires = routine.new_wires(n)
    for i in range(n):
        H(wires[i])
    return routine

Hs = hadamard_generator(5)
Hs.display()

If the custom gate is defined by a matrix, then `AsbtractGate` can be used. 

In [ ]:
import numpy as np
from qat.lang.AQASM import AbstractGate 

def L_generator(n): #circulant matrix
    N = 2**n-1
    M = np.diag(np.ones(N),k=-1)
    M[0,N]=1 
    return M
L = AbstractGate("L", 
                  [int], #type of the parameter
                  arity=lambda n: n, #arity is the number of qubits
                  matrix_generator = L_generator
                  )  

L_test = Program()
qubits = L_test.qalloc(4)
L(3)(qubits[0],qubits[2],qubits[3]) #C-Y gate
L_test.to_circ().display()